# TPI Database creation 

This notebook creates the SQL database for the TPI data. 

# Importing Necessary Downloads

In [2]:
import sys
import os
import re
import pandas as pd 
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import pandas as pd
import os
import re
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
# Add the project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Now import with full module path
from utils.database_creation_utils import get_db_connection, get_engine

engine = get_engine()



# Sector Entities 

We began by creating the sector table as it was the least connected to the other entities and therefore it is the logical starting point. 

In [5]:
df = pd.read_csv("../data/TPI_sector_data_All_sectors_08032025/Sector_Benchmarks_08032025.csv")
df.columns = df.columns.str.strip()

# Drop and create tables
create_tables_sql = """
DROP TABLE IF EXISTS benchmark_projection;
DROP TABLE IF EXISTS sector_benchmark;

CREATE TABLE sector_benchmark (
  benchmark_id VARCHAR NOT NULL,
  sector_name VARCHAR NOT NULL,
  scenario_name VARCHAR NOT NULL,
  region VARCHAR NOT NULL,
  release_date DATE NOT NULL,
  unit VARCHAR NOT NULL,
  PRIMARY KEY (benchmark_id, sector_name, scenario_name)
);

CREATE TABLE benchmark_projection (
  benchmark_projection_year INT NOT NULL,
  benchmark_projection_attribute FLOAT,
  benchmark_id VARCHAR NOT NULL,
  sector_name VARCHAR NOT NULL,
  scenario_name VARCHAR NOT NULL,
  FOREIGN KEY (benchmark_id, sector_name, scenario_name) 
    REFERENCES sector_benchmark(benchmark_id, sector_name, scenario_name)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_tables_sql))
    conn.commit()

# Prepare Sector_Benchmark data
sector_benchmark_df = df[['Benchmark ID', 'Sector name', 'Scenario name', 'Region', 'Release date', 'Unit']].drop_duplicates()
sector_benchmark_df.columns = ['benchmark_id', 'sector_name', 'scenario_name', 'region', 'release_date', 'unit']
sector_benchmark_df['release_date'] = pd.to_datetime(sector_benchmark_df['release_date'], dayfirst=True)

# Create composite key for FK matching
sector_benchmark_df['benchmark_key'] = (
    sector_benchmark_df['benchmark_id'] + '||' +
    sector_benchmark_df['sector_name'] + '||' +
    sector_benchmark_df['scenario_name']
)

# Insert into sector_benchmark
sector_benchmark_df.drop(columns=['benchmark_key']).to_sql('sector_benchmark', engine, if_exists='append', index=False)

# Prepare benchmark_projection
projection_years = [str(y) for y in range(2013, 2051)]
benchmark_projection_df = df.melt(
    id_vars=['Benchmark ID', 'Sector name', 'Scenario name'],
    value_vars=projection_years,
    var_name='benchmark_projection_year',
    value_name='benchmark_projection_attribute'
)
benchmark_projection_df.columns = ['benchmark_id', 'sector_name', 'scenario_name', 'benchmark_projection_year', 'benchmark_projection_attribute']
benchmark_projection_df['benchmark_projection_year'] = benchmark_projection_df['benchmark_projection_year'].astype(int)

# Create composite key for join check
benchmark_projection_df['benchmark_key'] = (
    benchmark_projection_df['benchmark_id'] + '||' +
    benchmark_projection_df['sector_name'] + '||' +
    benchmark_projection_df['scenario_name']
)

# Enforce FK integrity
valid_keys = sector_benchmark_df['benchmark_key'].unique()
benchmark_projection_df = benchmark_projection_df[benchmark_projection_df['benchmark_key'].isin(valid_keys)]

# Drop rows with missing projection values
benchmark_projection_df = benchmark_projection_df.dropna(subset=['benchmark_projection_attribute'])

# Final insert
benchmark_projection_df.drop(columns=['benchmark_key'], inplace=True)
benchmark_projection_df.to_sql('benchmark_projection', engine, if_exists='append', index=False)

print("Tables created and populated successfully.")


Tables created and populated successfully.


# Company Entities

In [10]:

# Define paths to company files
file_5 = "../data/TPI_sector_data_All_sectors_08032025/Company_Latest_Assessments_5.0.csv"
file_4 = "../data/TPI_sector_data_All_sectors_08032025/Company_Latest_Assessments.csv"

# Load the files
df_5 = pd.read_csv(file_5)
df_4 = pd.read_csv(file_4)

# Define and run SQL to create the tables
create_company_tables_sql = """
DROP TABLE IF EXISTS company_answer;
DROP TABLE IF EXISTS company;

CREATE TABLE company (
  company_name VARCHAR NOT NULL,
  geography VARCHAR,
  isin VARCHAR,
  ca100_focus VARCHAR,
  size_classification VARCHAR,
  geography_code VARCHAR,
  sedol VARCHAR,
  version VARCHAR NOT NULL,
  sector_name VARCHAR,
  PRIMARY KEY (company_name, version)
);

CREATE TABLE company_answer (
  question_code VARCHAR NOT NULL,
  question_text VARCHAR NOT NULL,
  response VARCHAR NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  PRIMARY KEY (question_code, company_name, version),
  FOREIGN KEY (company_name, version) REFERENCES company(company_name, version)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_company_tables_sql))
    conn.commit()

# Map metadata columns
meta_cols_common = {
    'Company Name': 'company_name',
    'Geography': 'geography',
    'Geography Code': 'geography_code',
    'Sector': 'sector_name',
    'CA100 Focus Company': 'ca100_focus',
    'Large/Medium Classification': 'size_classification',
    'ISINs': 'isin',
    'SEDOL': 'sedol'
}

# Extract and tag company data
df_5_meta = df_5[list(meta_cols_common.keys())].rename(columns=meta_cols_common)
df_4_meta = df_4[list(meta_cols_common.keys())].rename(columns=meta_cols_common)
df_5_meta["version"] = "5.0"
df_4_meta["version"] = "4.0"
company_df = pd.concat([df_4_meta, df_5_meta], ignore_index=True)

# Insert into company table
company_df.to_sql("company", engine, if_exists="append", index=False)

# Build company_answer DataFrame
def extract_answers(df, version):
    questions = [col for col in df.columns if col.startswith("Q") and "|" in col]
    records = []
    for q in questions:
        code, text = q.split("|", 1)
        for _, row in df.iterrows():
            records.append({
                "question_code": code.strip(),
                "question_text": text.strip(),
                "response": row[q],
                "company_name": row["Company Name"].strip(),
                "version": version
            })
    return pd.DataFrame(records)

answers_4 = extract_answers(df_4, "4.0")
answers_5 = extract_answers(df_5, "5.0")
company_answer_df = pd.concat([answers_4, answers_5], ignore_index=True)

# Drop null responses (required for NOT NULL constraint)
company_answer_df = company_answer_df.dropna(subset=["response"])

# Enforce FK constraint: only include answers for existing companies
valid_keys = set(zip(company_df['company_name'], company_df['version']))
company_answer_df = company_answer_df[
    company_answer_df.apply(lambda row: (row['company_name'], row['version']) in valid_keys, axis=1)
]

# Insert into company_answer table
company_answer_df.to_sql("company_answer", engine, if_exists="append", index=False)

print("✅ Company and answer tables created and populated successfully.")


InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table company because other objects depend on it
DETAIL:  constraint cp_assessment_company_name_version_fkey on table cp_assessment depends on table company
constraint mq_assessment_company_name_version_fkey on table mq_assessment depends on table company
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE IF EXISTS company_answer;
DROP TABLE IF EXISTS company;

CREATE TABLE company (
  company_name VARCHAR NOT NULL,
  geography VARCHAR,
  isin VARCHAR,
  ca100_focus VARCHAR,
  size_classification VARCHAR,
  geography_code VARCHAR,
  sedol VARCHAR,
  version VARCHAR NOT NULL,
  sector_name VARCHAR,
  PRIMARY KEY (company_name, version)
);

CREATE TABLE company_answer (
  question_code VARCHAR NOT NULL,
  question_text VARCHAR NOT NULL,
  response VARCHAR NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  PRIMARY KEY (question_code, company_name, version),
  FOREIG

# MQ Assessments 

In [12]:

mq_dir = "../data/TPI_sector_data_All_sectors_08032025"

# Dynamically find MQ files and extract tpi_cycle
mq_files = {}
for fname in os.listdir(mq_dir):
    match = re.match(r"MQ_Assessments_Methodology_(\d+)_.*\.csv", fname)
    if match:
        cycle = int(match.group(1))
        mq_files[cycle] = os.path.join(mq_dir, fname)

# Load latest company versions
with engine.connect() as conn:
    company_df = pd.read_sql("SELECT company_name, MAX(version) as version FROM company GROUP BY company_name", conn)

# Create MQ table
create_mq_sql = """
DROP TABLE IF EXISTS mq_assessment;

CREATE TABLE mq_assessment (
  assessment_date DATE,
  publication_date DATE,
  level INT,
  performance_change VARCHAR,
  tpi_cycle INT NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  PRIMARY KEY (company_name, version, tpi_cycle, assessment_date),
  FOREIGN KEY (company_name, version) REFERENCES company(company_name, version)
);
"""
with engine.connect() as conn:
    conn.execute(text(create_mq_sql))
    conn.commit()

# Process all MQ files
mq_records = []
for cycle, path in mq_files.items():
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    df['tpi_cycle'] = cycle
    df['company_name'] = df['Company Name'].str.strip()
    df['assessment_date'] = pd.to_datetime(df['Assessment Date'], dayfirst=True, errors='coerce')
    df['publication_date'] = pd.to_datetime(df['Publication Date'], errors='coerce')
    df['level'] = pd.to_numeric(df['Level'], errors='coerce')
    df['performance_change'] = df['Performance compared to previous year'].astype(str)
    df = df[['company_name', 'assessment_date', 'publication_date', 'level', 'performance_change', 'tpi_cycle']]
    mq_records.append(df)

mq_df = pd.concat(mq_records, ignore_index=True)

# Merge with most recent company version
mq_df = mq_df.merge(company_df, on='company_name', how='inner')

# Final insert
mq_df.to_sql("mq_assessment", engine, if_exists="append", index=False)

print("✅ MQ assessments table created and populated.")


✅ MQ assessments table created and populated.


# CP Assessments

In [13]:
cp_dir = "../data/TPI_sector_data_All_sectors_08032025"


# Step 1: Create CP tables
create_cp_sql = """
DROP TABLE IF EXISTS cp_projection;
DROP TABLE IF EXISTS cp_alignment;
DROP TABLE IF EXISTS cp_assessment;

CREATE TABLE cp_assessment (
  assessment_date DATE NOT NULL,
  publication_date DATE,
  assumptions VARCHAR,
  cp_unit VARCHAR,
  projection_cutoff DATE,
  benchmark_id VARCHAR,
  is_regional VARCHAR NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  PRIMARY KEY (assessment_date, is_regional, company_name, version),
  FOREIGN KEY (company_name, version) REFERENCES company(company_name, version)
);

CREATE TABLE cp_alignment (
  cp_alignment_year INT NOT NULL,
  cp_alignment_value VARCHAR NOT NULL,
  assessment_date DATE NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  is_regional VARCHAR NOT NULL,
  FOREIGN KEY (assessment_date, company_name, version, is_regional)
    REFERENCES cp_assessment(assessment_date, company_name, version, is_regional)
);

CREATE TABLE cp_projection (
  cp_projection_year INT NOT NULL,
  cp_projection_value INT NOT NULL,
  assessment_date DATE NOT NULL,
  company_name VARCHAR NOT NULL,
  version VARCHAR NOT NULL,
  is_regional VARCHAR NOT NULL,
  FOREIGN KEY (assessment_date, company_name, version, is_regional)
    REFERENCES cp_assessment(assessment_date, company_name, version, is_regional)
);
"""
with engine.connect() as conn:
    conn.execute(text(create_cp_sql))
    conn.commit()

# Step 2: Load company versions
with engine.connect() as conn:
    company_df = pd.read_sql("SELECT company_name, MAX(version) as version FROM company GROUP BY company_name", conn)

# Step 3: Dynamically find CP files
cp_files = {}
for fname in os.listdir(cp_dir):
    if fname.startswith("CP_Assessments_Regional"):
        cp_files["1"] = os.path.join(cp_dir, fname)
    elif fname.startswith("CP_Assessments"):
        cp_files["0"] = os.path.join(cp_dir, fname)

# Step 4: Process and insert data
assessment_records = []
alignment_records = []
projection_records = []

for is_regional, path in cp_files.items():
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    df["company_name"] = df["Company Name"].str.strip()
    df["assessment_date"] = pd.to_datetime(df["Assessment Date"], dayfirst=True, errors='coerce')
    df["publication_date"] = pd.to_datetime(df["Publication Date"], errors='coerce')
    df["projection_cutoff"] = pd.to_datetime(df["History to Projection cutoff year"], errors='coerce')
    df["assumptions"] = df.get("Assumptions")
    df["cp_unit"] = df["CP Unit"]
    df["benchmark_id"] = df.get("Benchmark ID")
    df["is_regional"] = is_regional

    assessment_df = df[[
        "company_name", "assessment_date", "publication_date",
        "assumptions", "cp_unit", "projection_cutoff", "benchmark_id", "is_regional"
    ]].merge(company_df, on="company_name", how="inner")

    assessment_records.append(assessment_df)

    # CP Alignment columns
    align_cols = [col for col in df.columns if col.startswith("Carbon Performance Alignment ")]
    for col in align_cols:
        year = int(col.split()[-1])
        temp = df[["company_name", "assessment_date"]].copy()
        temp["cp_alignment_year"] = year
        temp["cp_alignment_value"] = df[col]
        temp["is_regional"] = is_regional
        temp = temp.merge(company_df, on="company_name", how="inner")
        temp = temp.dropna(subset=["cp_alignment_value"])
        alignment_records.append(temp)

    # CP Projection columns
    year_cols = [col for col in df.columns if re.fullmatch(r"\d{4}", col)]
    for col in year_cols:
        year = int(col)
        temp = df[["company_name", "assessment_date"]].copy()
        temp["cp_projection_year"] = year
        temp["cp_projection_value"] = df[col]
        temp["is_regional"] = is_regional
        temp = temp.merge(company_df, on="company_name", how="inner")
        temp = temp.dropna(subset=["cp_projection_value"])
        projection_records.append(temp)


# Step 5: Insert into DB
pd.concat(assessment_records).to_sql("cp_assessment", engine, if_exists="append", index=False)
pd.concat(alignment_records).to_sql("cp_alignment", engine, if_exists="append", index=False)
pd.concat(projection_records).to_sql("cp_projection", engine, if_exists="append", index=False)
print("✅ CP tables created and populated successfully.")


✅ CP tables created and populated successfully.
